In [1]:
import pandas as pd
import json

/Users/aaravnair/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/aaravnair/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
population_df = pd.read_excel('../../data/landing/abs_population.xlsx', sheet_name='Table 1', skiprows=6, skipfooter=5)
income_df = pd.read_excel('../../data/landing/abs_income.xlsx', sheet_name='Table 1', skiprows=6, skipfooter=5)

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [ ]:
with open('../../data/landing/postcode_mapping.json') as f:
    postcode_mapping = json.load(f)

f.close()

suburb_names = set([suburb for sublist in postcode_mapping.values() for suburb in sublist])

In [ ]:
sa2_df = pd.read_csv('../../data/curated/SA2_Victoria.csv')
sa2_names = sa2_df['SA2_NAME_2021']

In [ ]:
population_df = population_df[population_df['Label'].isin(sa2_names)]
population_df.dropna(axis=1, how='all', inplace=True)

income_df = income_df[income_df['Label'].isin(sa2_names)]
income_df.dropna(axis=1, how='all', inplace=True)

population_df = population_df[['Label', 'Year', 'Estimated resident population (no.)', 'Population density (persons/km2)', 'Median age - persons (years)', 'Total fertility rate (births per female) (rate)', 'Standardised death rate (per 1000 people) (rate)']]
population_df.dropna(axis=0, how='any', inplace=True)

income_df = income_df[['Label', 'Year', 'Median employee income ($)']]
income_df.dropna(axis=0, how='any', inplace=True)

population_df = population_df[population_df['Year'] >= 2018]
population_df = population_df[population_df['Year'] <= 2023]

# The 5 years that we have data for
income_df = income_df[income_df['Year'] >= 2016]
income_df = income_df[income_df['Year'] <= 2020]

population_df['Year'] = population_df['Year'].astype(int)
income_df['Year'] = income_df['Year'].astype(int)

In [ ]:
population_df[population_df['Label'] == 'Carlton']

,Label,Year,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Total fertility rate (births per female) (rate),Standardised death rate (per 1000 people) (rate)
11019,Carlton,2018,20831,11453.8,25.7,0.82,4.6
11020,Carlton,2019,21029,11562.7,26.3,0.68,5.2
11021,Carlton,2020,20865,11472.5,27.1,0.55,4.9
11022,Carlton,2021,17064,9382.5,27.8,0.54,5.2
11023,Carlton,2022,18057,9928.5,26.9,0.56,5.7
11024,Carlton,2023,21376,11753.5,-,-,-


In [ ]:
income_df[income_df['Label'] == 'Carlton']

,Label,Year,Median employee income ($)
9793,Carlton,2016,30001
9794,Carlton,2017,28320
9795,Carlton,2018,30698
9796,Carlton,2019,31642
9797,Carlton,2020,33346


In [ ]:
# Remove duplicate rows based on 'Label' and 'Year'
population_df = population_df.drop_duplicates(subset=['Label', 'Year'])
income_df = income_df.drop_duplicates(subset=['Label', 'Year'])

population_df = population_df.pivot(
    index='Label', 
    columns='Year', 
    values='Estimated resident population (no.)'
)

income_df = income_df.pivot(
    index='Label',
    columns='Year',
    values='Median employee income ($)'
)

# Rename the columns to make them more readable
population_df.columns = [f'estimated_population_{int(col)}' for col in population_df.columns]
income_df.columns = [f'median_income_{int(col)}' for col in income_df.columns]

In [ ]:
population_df[population_df.index == 'Carlton']

,estimated_population_2018,estimated_population_2019,estimated_population_2020,estimated_population_2021,estimated_population_2022,estimated_population_2023
Label,,,,,,
Carlton,20831,21029,20865,17064,18057,21376


In [ ]:
income_df[income_df.index == 'Carlton']

,median_income_2016,median_income_2017,median_income_2018,median_income_2019,median_income_2020
Label,,,,,
Carlton,30001,28320,30698,31642,33346


In [ ]:
# Replace '-' with NaN
population_df.replace('-', None, inplace=True)
income_df.replace('-', None, inplace=True)

# Drop rows with any NaN values (which were previously '-')
population_df.dropna(inplace=True)
income_df.dropna(inplace=True)

# Select the population columns (excluding 'Label') and convert to int
population_columns = [col for col in population_df.columns if col.startswith('Estimated_population_')]
income_columns = [col for col in income_df.columns if col.startswith('Median_income_')]

# Convert the selected columns to int/float
population_df[population_columns] = population_df[population_columns].astype(int)
income_df[income_columns] = income_df[income_columns].astype(int)

In [ ]:
population_df

,estimated_population_2018,estimated_population_2019,estimated_population_2020,estimated_population_2021,estimated_population_2022,estimated_population_2023
Label,,,,,,
Abbotsford,9527,9594,9672,9258,9513,10008
Airport West,8169,8390,8362,8240,8295,8464
Albert Park,16728,17081,16955,16011,16177,16861
Alexandra,6646,6687,6690,6771,6794,6836
Alfredton,13537,14434,15507,16841,18002,18997
...,...,...,...,...,...,...
Yarram,5437,5474,5545,5555,5588,5580
Yarraville,15991,16092,16068,15651,15661,16020
Yarrawonga,8297,8418,8508,8593,8727,8812


In [ ]:
income_df

,median_income_2016,median_income_2017,median_income_2018,median_income_2019,median_income_2020
Label,,,,,
Abbotsford,56001,56242,59923,61918,66091
Airport West,54708,56619,58350,60992,62920
Albert Park,64048,62922,64900,65880,68836
Alexandra,38251,39811,41230,42447,44033
Alfredton,49071,50119,52709,53765,55019
...,...,...,...,...,...
Yarram,40266,39960,42650,43228,44904
Yarraville,61906,63996,66465,69379,72428
Yarrawonga,40234,40400,41809,43914,45708


In [14]:
population_df.to_parquet('../../data/curated/population_by_sa2.parquet', index=True)
income_df.to_parquet('../../data/curated/income_by_sa2.parquet', index=True)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [16]:
# Prepare the feature data (2019-2022 population data) and target (growth rate for 2023)
X = population_df[['estimated_population_2018', 'estimated_population_2019', 'estimated_population_2020', 'estimated_population_2021', 'estimated_population_2022']].rename(columns={'estimated_population_2018': '1', 'estimated_population_2019': '2', 'estimated_population_2020': '3', 'estimated_population_2021': '4', 'estimated_population_2022': '5'})
y = population_df['estimated_population_2023'].rename('6')

# Split into train and test sets (test set will be the data for 2023)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30034)

In [17]:
from sklearn.linear_model import LinearRegression

# Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict on the test set (2023)
lr_predictions = lr_model.predict(X_test)

In [18]:
from sklearn.ensemble import RandomForestRegressor

# Train the Random Forest model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Predict on the test set (2023)
rf_predictions = rf_model.predict(X_test)

In [19]:
from sklearn.ensemble import GradientBoostingRegressor

# Train the Gradient Boosting model
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

# Predict on the test set (2023)
gb_predictions = gb_model.predict(X_test)

In [20]:
# Define a function to calculate RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Calculate RMSE for each model
lr_rmse = calculate_rmse(y_test, lr_predictions)
rf_rmse = calculate_rmse(y_test, rf_predictions)
gb_rmse = calculate_rmse(y_test, gb_predictions)

print(f"Linear Regression RMSE: {lr_rmse}")
print(f"Random Forest RMSE: {rf_rmse}")
print(f"Gradient Boosting RMSE: {gb_rmse}")

Linear Regression RMSE: 198.152022644975
Random Forest RMSE: 2182.676104379183
Gradient Boosting RMSE: 2189.2788462351577


In [21]:
population_df['estimated_population_2024'] = lr_model.predict(population_df[['estimated_population_2019', 'estimated_population_2020', 'estimated_population_2021', 'estimated_population_2022', 'estimated_population_2023']].rename(columns={'estimated_population_2019': '1', 'estimated_population_2020': '2', 'estimated_population_2021': '3', 'estimated_population_2022': '4', 'estimated_population_2023': '5'}))
population_df['estimated_population_2025'] = lr_model.predict(population_df[['estimated_population_2020', 'estimated_population_2021', 'estimated_population_2022', 'estimated_population_2023', 'estimated_population_2024']].rename(columns={'estimated_population_2020': '1', 'estimated_population_2021': '2', 'estimated_population_2022': '3', 'estimated_population_2023': '4', 'estimated_population_2024': '5'}))
population_df['estimated_population_2026'] = lr_model.predict(population_df[['estimated_population_2021', 'estimated_population_2022', 'estimated_population_2023', 'estimated_population_2024', 'estimated_population_2025']].rename(columns={'estimated_population_2021': '1', 'estimated_population_2022': '2', 'estimated_population_2023': '3', 'estimated_population_2024': '4', 'estimated_population_2025': '5'}))

In [30]:
population_df.to_csv('../../data/curated/population_by_sa2.csv', index=True)
population_df.isna().sum()

estimated_population_2018    0
estimated_population_2019    0
estimated_population_2020    0
estimated_population_2021    0
estimated_population_2022    0
estimated_population_2023    0
estimated_population_2024    0
estimated_population_2025    0
estimated_population_2026    0
dtype: int64

In [23]:
X = income_df[['median_income_2016', 'median_income_2017', 'median_income_2018', 'median_income_2019']].rename(columns={'median_income_2016': '1', 'median_income_2017': '2', 'median_income_2018': '3', 'median_income_2019': '4'})
y = income_df['median_income_2020'].rename('5')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30034)

In [24]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

lr_predictions = lr_model.predict(X_test)

lr_rmse = calculate_rmse(y_test, lr_predictions)
print(f"Linear Regression RMSE: {lr_rmse}")

Linear Regression RMSE: 2318.8537430069355


In [25]:
income_df['median_income_2021'] = lr_model.predict(income_df[['median_income_2017', 'median_income_2018', 'median_income_2019', 'median_income_2020']].rename(columns={'median_income_2017': '1', 'median_income_2018': '2', 'median_income_2019': '3', 'median_income_2020': '4'}))
income_df['median_income_2022'] = lr_model.predict(income_df[['median_income_2018', 'median_income_2019', 'median_income_2020', 'median_income_2021']].rename(columns={'median_income_2018': '1', 'median_income_2019': '2', 'median_income_2020': '3', 'median_income_2021': '4'}))
income_df['median_income_2023'] = lr_model.predict(income_df[['median_income_2019', 'median_income_2020', 'median_income_2021', 'median_income_2022']].rename(columns={'median_income_2019': '1', 'median_income_2020': '2', 'median_income_2021': '3', 'median_income_2022': '4'}))
income_df['median_income_2024'] = lr_model.predict(income_df[['median_income_2020', 'median_income_2021', 'median_income_2022', 'median_income_2023']].rename(columns={'median_income_2020': '1', 'median_income_2021': '2', 'median_income_2022': '3', 'median_income_2023': '4'}))
income_df['median_income_2025'] = lr_model.predict(income_df[['median_income_2021', 'median_income_2022', 'median_income_2023', 'median_income_2024']].rename(columns={'median_income_2021': '1', 'median_income_2022': '2', 'median_income_2023': '3', 'median_income_2024': '4'}))
income_df['median_income_2026'] = lr_model.predict(income_df[['median_income_2022', 'median_income_2023', 'median_income_2024', 'median_income_2025']].rename(columns={'median_income_2022': '1', 'median_income_2023': '2', 'median_income_2024': '3', 'median_income_2025': '4'}))

In [31]:
income_df.to_csv('../../data/curated/income_by_sa2.csv', index=True)
income_df.isna().sum()

median_income_2016    0
median_income_2017    0
median_income_2018    0
median_income_2019    0
median_income_2020    0
median_income_2021    0
median_income_2022    0
median_income_2023    0
median_income_2024    0
median_income_2025    0
median_income_2026    0
dtype: int64